This example will walk you throught the basic usage of PromptBench. We hope that you can get familiar with the APIs and use it in your own projects later.

First, there is a unified import of `import promptbench as pb` that easily imports the package.

In [2]:
import promptbench as pb

## Load dataset

First, PromptBench supports easy load of datasets.

In [3]:
# print all supported datasets in promptbench
print(pb.DatasetLoader.dataset_list())

# load a dataset, sst2, for instance.
# if the dataset is not available locally, it will be downloaded automatically.
dataset = pb.DatasetLoader.load_dataset("sst2")

# print the first 5 examples
dataset[:5]

['cola', 'sst2', 'qqp', 'mnli', 'mnli_matched', 'mnli_mismatched', 'qnli', 'wnli', 'rte', 'mrpc', 'mmlu', 'squad_v2', 'un_multi', 'iwslt', 'math', 'bool_logic', 'valid_parentheses']


[{'content': "it 's a charming and often affecting journey . ", 'label': 1},
 {'content': 'unflinchingly bleak and desperate ', 'label': 0},
 {'content': 'allows us to hope that nolan is poised to embark a major career as a commercial yet inventive filmmaker . ',
  'label': 1},
 {'content': "the acting , costumes , music , cinematography and sound are all astounding given the production 's austere locales . ",
  'label': 1},
 {'content': "it 's slow -- very , very slow . ", 'label': 0}]

In [4]:
print(pb.LLMModel.model_list())
model = pb.LLMModel(model='google/flan-t5-large', max_new_tokens=10)

{'t5': ['google/flan-t5-large'], 'llama': ['llama2-7b', 'llama2-7b-chat', 'llama2-13b', 'llama2-13b-chat', 'llama2-70b', 'llama2-70b-chat'], 'gpt': ['gpt-3.5-turbo', 'gpt-4'], 'vicuna': ['vicuna-7b', 'vicuna-13b', 'vicuna-13b-v1.3'], 'ul2': ['google/flan-ul2']}



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /media/Zeus/haoc/miniconda/envs/promptbench/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda116.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 116
CUDA SETUP: Loading binary /media/Zeus/haoc/miniconda/envs/promptbench/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda116.so...


/media/Zeus/haoc/miniconda/envs/promptbench/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /media/Zeus/haoc/miniconda/envs/promptbench did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/media/Zeus/haoc/miniconda/envs/promptbench/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('vs/workbench/api/node/extensionHostProcess')}
  warn(msg)
/media/Zeus/haoc/miniconda/envs/promptbench/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('"/home/haoc/.vscode-server/data/clp/9f233b0c8bdef919c90298683f3dac6b.zh-cn","_resolvedLanguagePackCoreLocation"'), PosixPath('"zh-cn"},"_languagePackId"'), PosixPath('"/home/haoc/.vscode-server/data/clp/9f233b

In [5]:
prompts = pb.Prompt(["Classify the sentence as positive or negative: {content}",
                     "Determine the emotion of the following sentence as positive or negative: {content}"
                     ])

In [6]:
"""
You may need to define the projection function for the model output.
Since the output format defined in your prompts may be different from the model output.
For example, for sst2 dataset, the label are '0' and '1' to represent 'negative' and 'positive'.
But the model output is 'negative' and 'positive'.
So we need to define a projection function to map the model output to the label.
"""
def proj_func(pred):
    mapping = {
        "positive": 1,
        "negative": 0
    }
    return mapping.get(pred, -1)

In [7]:
from tqdm import tqdm
for prompt in prompts:
    preds = []
    labels = []
    for data in tqdm(dataset):
        # process input
        input_text = pb.InputProcess.basic_format(prompt, data)
        label = data['label']

        raw_pred = model(input_text)
        # process output
        pred = pb.OutputProcess.cls(raw_pred, proj_func)
        preds.append(pred)
        labels.append(label)
    
    # evaluate
    score = pb.Eval.compute_cls_accuracy(preds, labels)
    print(f"{score:.3f}, {prompt}")

100%|██████████| 872/872 [02:16<00:00,  6.37it/s]


0.947, Classify the sentence as positive or negative: {content}


100%|██████████| 872/872 [02:18<00:00,  6.29it/s]

0.947, Determine the emotion of the following sentence as positive or negative: {content}
